# fwd_bkw

In [25]:
from sklearn.preprocessing import normalize

from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import multivariate_normal

def fit(Y,K,p,labels,max_iter):
    # p is the degree of the polynomial regression
    # K is the number of classes
    #Y is a n*d obeservation matrix
    #labels is the labels matrix of Y
    """ Initialisation """
    pf = PolynomialFeatures(degree=p) #transformer to get polynomial features of the time
    t = pf.fit_transform(np.arange(n).reshape(-1,1)) #matrix containing the polynomial features of the time
    n,d = Y.shape
    pi = (1.0/K)*np.ones((K))
    A = (1.0/(2.0*(K-1)))*np.ones((K,K))
    for i in range(K):
        A[i,i] = 0.5
    B = [] #list of coefficient matrices
    sigma = [] #list of covariance matrices 
    for i in range(K):
        B[i] = np.random.uniform(-1,1,(p+1,d))
        sigma[i] = np.cov(np.transpose(Y[labels==i,:]))
    F = np.zeros((n,K))
    for i in range(n):
        for j in range(K):
            mean = np.dot(B,t[i,:])
            F[i,j] = multivariate_normal(mean=mean,cov=sigma[i]).pdf(Y[i,:])
    for i in range(max_iter):
        """ E step """
        #calculate P1 and P2
        #P1 is a n*K matrix
        #P2 is a matrix of size (n-1)*K*K
        P11,P2 = fwd_bkw(pi, F.T,A)
        P1 = P11.T
        """ M step """
        pi = P1[0,:]
        A = np.transpose(np.sum(A[1:,:,:],axis=0))
        for i in range(K):
            A[i,:] = A[i,:]/np.sum(P1[1:,:],axis=0)
        #calculate the Bs and the sigmas
        """ Update F """
        for i in range(n):
            for j in range(K):
                mean = np.dot(B,t[i,:])
                F[i,j] = multivariate_normal(mean=mean,cov=sigma[i]).pdf(Y[i,:])
    return pi,A,



def fwd_bkw(init_dist,F,A):
    [K,T] = F.shape
    alpha = np.zeros((K,T))
    beta = np.zeros((K,T))
    
    # Forward recursion
    t=1
    alpha[:,t] = np.multiply(init_dist,F[:,t])
    alpha[:,t] = normalize(alpha[:,t].reshape(-1,1),norm='l1')
    
    for t in np.arange(2,T+1):
        m = np.dot(A.T,alpha[:,t-1])
        alpha[:,t]=np.multiply(m,F[:,t])
        alpha[:,t] = normalize(alpha[:,t].reshape(-1,1),norm='l1')
    
    # Backward recursion
    beta[:,T] = normalize(np.ones(K).reshape(-1,1),norm='l1')
    for t in np.arange(1,T)[::-1]:
        b = np.multiply(beta[:,t+1], F[:,t+1])
        beta[:,t] = np.dot(A,b)
        beta[:,t] = normalize(beta[:,t].reshape(-1,1),norm='l1')
    
    # Compute P1=p(kt|y1,...,yT)=alpha*beta/sum(alpha*beta)
    P1 = np.zeros((T,K))
    for t in np.arange(T):
        P1[:,t] = np.multiply(alpha[:,t],beta[:,t])/np.sum(np.multiply(alpha[:,t],beta[:,t]))
        P1[:,t] = normalize(P1[:,t].reshape(-1,1),norm='l1')
    
    # Compute P2=p(kt,kt+1|y1,...,yT)
    P2 = np.zeros((T-1,K,K))
    for t in np.arange(1,T-2):
        for i in np.arange(1,K-1):
            for j in np.arange(1,K-1):
                P2[t,i,j]=alpha[i,t]*beta[j,t+1]*F[j,t+1]
        P2[t,:,:] = normalize(P2[t,:,:],norm='l1')
    return P1,P2